In [1]:
import wandb

import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist


(trainIn, trainOut), (testIn, testOut) = fashion_mnist.load_data()

N_train_full = trainOut.shape[0]
N_train = int(0.9*N_train_full)
N_validation = int(0.1 * trainOut.shape[0])
N_test = testOut.shape[0]


idx  = np.random.choice(trainOut.shape[0], N_train_full, replace=False)
idx2 = np.random.choice(testOut.shape[0], N_test, replace=False)

trainInFull = trainIn[idx, :]
trainOutFull = trainOut[idx]

trainIn = trainInFull[:N_train,:]
trainOut = trainOutFull[:N_train]

validIn = trainInFull[N_train:, :]
validOut = trainOutFull[N_train:]    

testIn = testIn[idx2, :]
testOut = testOut[idx2]

In [2]:
sweep_config = {
  "name": "Bayesian Sweep",
  "method": "bayes",
  "metric":{
  "name": "validationaccuracy",
  "goal": "maximize"
  },
  "parameters": {
        "max_epochs": {"values": [5, 10]},

        "hidden_layers": {"values": [3, 4, 5]},
        
        "no_of_hidden_neurons": {"values": [32, 64, 128]},
        
        "weight_decay": {"values": [0, 0.0005,0.5]},
        
        "learning_rate": {"values": [0.001, 0.0001]},
        
        "optimizer": {"values": ["SGD", "MGD", "NAG", "RMSPROP", "ADAM","NADAM"]},
                    
        "batch_size": {"values": [16, 32, 64]},

        "initializer": {"values": ["RANDOM", "XAVIER"]},
        
        "activation": {"values": [ 'tanh',  'sigmoid', 'ReLU']}
        }
    }

In [5]:
sweep_id = wandb.sweep(sweep_config,project='fashion-mnist-visualization')


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: mfnwfsvf
Sweep URL: https://wandb.ai/ma23c011-indian-institute-of-technology-madras/fashion-mnist-visualization/sweeps/mfnwfsvf


In [3]:
from neural_network import NeuralNetwork
def train():    
    config_defaults = dict(
            max_epochs=5,
            hidden_layers=3,
            no_of_hidden_neurons=32,
            weight_decay=0,
            learning_rate=1e-3,
            optimizer="NADAM",
            batch_size=32,
            activation="tanh",
            initializer="XAVIER",
            loss="CROSS",
        )
        
    wandb.init(config = config_defaults)


    wandb.run.name = "hl_" + str(wandb.config.hidden_layers) + "_hn_" + str(wandb.config.no_of_hidden_neurons) + "_opt_" + wandb.config.optimizer + "_act_" + wandb.config.activation + "_lr_" + str(wandb.config.learning_rate) + "_bs_"+str(wandb.config.batch_size) + "_init_" + wandb.config.initializer + "_ep_"+ str(wandb.config.max_epochs)+ "_l2_" + str(wandb.config.weight_decay) 
    CONFIG = wandb.config
    
    FFNN = NeuralNetwork(
        hidden_layers=CONFIG.hidden_layers,
        no_of_hidden_neurons=CONFIG.no_of_hidden_neurons,
        train_In=trainIn,
        train_out=trainOut,
        N_train = N_train,
        X_val_In = validIn,
        val_out = validOut,
        N_val = N_validation,
        test_In = testIn,
        test_out = testOut,
        N_test = N_test,
        optimizer = CONFIG.optimizer,
        batch_size = CONFIG.batch_size,
        weight_decay = CONFIG.weight_decay,
        learning_rate = CONFIG.learning_rate,
        max_epochs = CONFIG.max_epochs,
        activation = CONFIG.activation,
        initializer = CONFIG.initializer,
        loss = CONFIG.loss
        )


    training_loss, training_accuracy, validation_loss, validation_accuracy, Y_pred = FFNN.optimizer(FFNN.max_epochs, FFNN.N_train, FFNN.batch_size, FFNN.learning_rate)
 

In [18]:
wandb.agent(sweep_id, train, count = 1)

wandb: Agent Starting Run: n3ap4l79 with config:
wandb: 	activation: ReLU
wandb: 	batch_size: 64
wandb: 	initializer: XAVIER
wandb: 	learning_rate: 0.001
wandb: 	max_epochs: 5
wandb: 	num_hidden_neurons: 32
wandb: 	num_layers: 3
wandb: 	optimizer: MGD
wandb: 	weight_decay: 0.5


Epoch: 0, Loss: 1.137e-01, Training accuracy:0.755, Validation Accuracy: 0.748, Time: 21.384, Learning Rate: 1.000e-03
Epoch: 1, Loss: 6.240e-02, Training accuracy:0.804, Validation Accuracy: 0.802, Time: 23.699, Learning Rate: 1.000e-03
Epoch: 2, Loss: 5.411e-02, Training accuracy:0.823, Validation Accuracy: 0.821, Time: 25.473, Learning Rate: 1.000e-03
Epoch: 3, Loss: 5.014e-02, Training accuracy:0.833, Validation Accuracy: 0.830, Time: 28.736, Learning Rate: 1.000e-03
Epoch: 4, Loss: 4.764e-02, Training accuracy:0.839, Validation Accuracy: 0.838, Time: 25.685, Learning Rate: 1.000e-03


epoch,▁▃▅▆█
loss,█▃▂▁▁
training_accuracy,▁▅▇▇█
validation_accuracy,▁▅▇▇█
epoch,4
loss,0.04764
training_accuracy,0.83935
validation_accuracy,0.838


In [19]:
wandb.agent(sweep_id, train, count = 5)

wandb: Agent Starting Run: yke4gnku with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 16
wandb: 	initializer: XAVIER
wandb: 	learning_rate: 0.001
wandb: 	max_epochs: 10
wandb: 	num_hidden_neurons: 64
wandb: 	num_layers: 5
wandb: 	optimizer: NADAM
wandb: 	weight_decay: 0


Epoch: 0, Loss: 8.968e-02, Training accuracy:0.801, Validation Accuracy: 0.801, Time: 26.821, Learning Rate: 1.000e-03
Epoch: 1, Loss: 4.906e-02, Training accuracy:0.850, Validation Accuracy: 0.841, Time: 35.559, Learning Rate: 1.000e-03
Epoch: 2, Loss: 4.226e-02, Training accuracy:0.862, Validation Accuracy: 0.854, Time: 24.274, Learning Rate: 1.000e-03
Epoch: 3, Loss: 3.900e-02, Training accuracy:0.870, Validation Accuracy: 0.862, Time: 29.098, Learning Rate: 1.000e-03
Epoch: 4, Loss: 3.687e-02, Training accuracy:0.877, Validation Accuracy: 0.868, Time: 26.231, Learning Rate: 1.000e-03
Epoch: 5, Loss: 3.533e-02, Training accuracy:0.880, Validation Accuracy: 0.872, Time: 26.075, Learning Rate: 1.000e-03
Epoch: 6, Loss: 3.413e-02, Training accuracy:0.883, Validation Accuracy: 0.875, Time: 24.872, Learning Rate: 1.000e-03
Epoch: 7, Loss: 3.316e-02, Training accuracy:0.886, Validation Accuracy: 0.875, Time: 27.754, Learning Rate: 1.000e-03
Epoch: 8, Loss: 3.235e-02, Training accuracy:0.8

epoch,▁▂▃▃▄▅▆▆▇█
loss,█▃▂▂▂▁▁▁▁▁
training_accuracy,▁▅▆▆▇▇▇███
validation_accuracy,▁▅▆▇▇█████
epoch,9
loss,0.03165
training_accuracy,0.88996
validation_accuracy,0.87417


wandb: Agent Starting Run: pq3tfnwf with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 16
wandb: 	initializer: RANDOM
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 10
wandb: 	num_hidden_neurons: 32
wandb: 	num_layers: 4
wandb: 	optimizer: MGD
wandb: 	weight_decay: 0.0005


Epoch: 0, Loss: 2.825e-01, Training accuracy:0.173, Validation Accuracy: 0.188, Time: 37.556, Learning Rate: 1.000e-04
Epoch: 1, Loss: 2.122e-01, Training accuracy:0.260, Validation Accuracy: 0.272, Time: 35.331, Learning Rate: 1.000e-04
Epoch: 2, Loss: 1.956e-01, Training accuracy:0.336, Validation Accuracy: 0.345, Time: 32.921, Learning Rate: 1.000e-04
Epoch: 3, Loss: 1.829e-01, Training accuracy:0.396, Validation Accuracy: 0.408, Time: 30.892, Learning Rate: 1.000e-04
Epoch: 4, Loss: 1.721e-01, Training accuracy:0.441, Validation Accuracy: 0.457, Time: 29.459, Learning Rate: 1.000e-04
Epoch: 5, Loss: 1.629e-01, Training accuracy:0.474, Validation Accuracy: 0.489, Time: 32.921, Learning Rate: 1.000e-04
Epoch: 6, Loss: 1.547e-01, Training accuracy:0.498, Validation Accuracy: 0.512, Time: 30.451, Learning Rate: 1.000e-04
Epoch: 7, Loss: 1.475e-01, Training accuracy:0.516, Validation Accuracy: 0.527, Time: 26.083, Learning Rate: 1.000e-04
Epoch: 8, Loss: 1.411e-01, Training accuracy:0.5

...

Epoch: 0, Loss: 1.360e-01, Training accuracy:0.696, Validation Accuracy: 0.689, Time: 31.402, Learning Rate: 1.000e-04
Epoch: 1, Loss: 8.643e-02, Training accuracy:0.752, Validation Accuracy: 0.746, Time: 31.318, Learning Rate: 1.000e-04
Epoch: 2, Loss: 7.182e-02, Training accuracy:0.783, Validation Accuracy: 0.774, Time: 24.942, Learning Rate: 1.000e-04
Epoch: 3, Loss: 6.354e-02, Training accuracy:0.799, Validation Accuracy: 0.794, Time: 25.857, Learning Rate: 1.000e-04
Epoch: 4, Loss: 5.831e-02, Training accuracy:0.809, Validation Accuracy: 0.804, Time: 22.927, Learning Rate: 1.000e-04


epoch,▁▃▅▆█
loss,█▄▂▁▁
training_accuracy,▁▄▆▇█
validation_accuracy,▁▄▆▇█
epoch,4
loss,0.05831
training_accuracy,0.80941
validation_accuracy,0.804


wandb: Agent Starting Run: w970y894 with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 64
wandb: 	initializer: XAVIER
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 5
wandb: 	num_hidden_neurons: 128
wandb: 	num_layers: 5
wandb: 	optimizer: NAG
wandb: 	weight_decay: 0.0005


Epoch: 0, Loss: 2.421e-01, Training accuracy:0.100, Validation Accuracy: 0.101, Time: 27.351, Learning Rate: 1.000e-04
Epoch: 1, Loss: 2.324e-01, Training accuracy:0.100, Validation Accuracy: 0.101, Time: 27.881, Learning Rate: 1.000e-04
Epoch: 2, Loss: 2.308e-01, Training accuracy:0.100, Validation Accuracy: 0.101, Time: 38.233, Learning Rate: 1.000e-04
Epoch: 3, Loss: 2.305e-01, Training accuracy:0.101, Validation Accuracy: 0.102, Time: 24.643, Learning Rate: 1.000e-04
Epoch: 4, Loss: 2.304e-01, Training accuracy:0.109, Validation Accuracy: 0.106, Time: 56.869, Learning Rate: 1.000e-04


epoch,▁▃▅▆█
loss,█▂▁▁▁
training_accuracy,▁▁▁▂█
validation_accuracy,▁▁▁▂█
epoch,4
loss,0.23036
training_accuracy,0.10861
validation_accuracy,0.106


In [24]:
wandb.agent(sweep_id, train, count = 1)

wandb: Agent Starting Run: vuepspuz with config:
wandb: 	activation: tanh
wandb: 	batch_size: 32
wandb: 	hidden_layers: 4
wandb: 	initializer: XAVIER
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 10
wandb: 	no_of_hidden_neurons: 64
wandb: 	optimizer: NADAM
wandb: 	weight_decay: 0.5


Epoch: 0, Loss: 7.263e-02, Training accuracy:0.834, Validation Accuracy: 0.831, Time: 30.097, Learning Rate: 1.000e-04
Epoch: 1, Loss: 4.531e-02, Training accuracy:0.853, Validation Accuracy: 0.845, Time: 35.721, Learning Rate: 1.000e-04
Epoch: 2, Loss: 4.135e-02, Training accuracy:0.860, Validation Accuracy: 0.854, Time: 40.846, Learning Rate: 1.000e-04
Epoch: 3, Loss: 3.922e-02, Training accuracy:0.866, Validation Accuracy: 0.859, Time: 38.524, Learning Rate: 1.000e-04
Epoch: 4, Loss: 3.778e-02, Training accuracy:0.869, Validation Accuracy: 0.861, Time: 65.374, Learning Rate: 1.000e-04
Epoch: 5, Loss: 3.667e-02, Training accuracy:0.872, Validation Accuracy: 0.863, Time: 49.864, Learning Rate: 1.000e-04
Epoch: 6, Loss: 3.578e-02, Training accuracy:0.875, Validation Accuracy: 0.865, Time: 37.249, Learning Rate: 1.000e-04
Epoch: 7, Loss: 3.501e-02, Training accuracy:0.877, Validation Accuracy: 0.866, Time: 31.135, Learning Rate: 1.000e-04
Epoch: 8, Loss: 3.435e-02, Training accuracy:0.8

epoch,▁▂▃▃▄▅▆▆▇█
loss,█▃▂▂▂▂▁▁▁▁
training_accuracy,▁▄▅▆▆▇▇▇██
validation_accuracy,▁▃▅▆▆▇▇▇██
epoch,9
loss,0.03374
training_accuracy,0.88096
validation_accuracy,0.86967


Run vuepspuz errored:
Traceback (most recent call last):
  File "c:\Users\himan\.conda\envs\hj\lib\site-packages\wandb\agents\pyagent.py", line 306, in _run_job
    self._function()
  File "C:\Users\himan\AppData\Local\Temp\ipykernel_25500\1770519332.py", line 45, in train
    training_loss, training_accuracy, validation_loss, validation_accuracy, Y_pred = FFNN.optimizer(FFNN.max_epochs, FFNN.N_train, FFNN.batch_size, FFNN.learning_rate)
ValueError: not enough values to unpack (expected 5, got 4)

wandb: ERROR Run vuepspuz errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "c:\Users\himan\.conda\envs\hj\lib\site-packages\wandb\agents\pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "C:\Users\himan\AppData\Local\Temp\ipykernel_25500\1770519332.py", line 45, in train
wandb: ERROR     training_loss, training_accuracy, validation_loss, validation_accuracy, Y_pred = FFNN.optimizer(FFNN.max_epochs, FFNN.N_train, FFNN.batch_siz

In [ ]:
wandb.agent(sweep_id, train, count = 1)

wandb: Agent Starting Run: uh4s27d9 with config:
wandb: 	activation: tanh
wandb: 	batch_size: 64
wandb: 	hidden_layers: 4
wandb: 	initializer: XAVIER
wandb: 	learning_rate: 0.001
wandb: 	max_epochs: 10
wandb: 	no_of_hidden_neurons: 128
wandb: 	optimizer: SGD
wandb: 	weight_decay: 0.0005
wandb: Currently logged in as: ma23c011 (ma23c011-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch: 0, Loss: 1.509e-01, Training accuracy:0.691, Validation Loss: 1.116e-01, Validation Accuracy: 0.696, Time: 113.437, Learning Rate: 1.000e-03
Epoch: 1, Loss: 9.757e-02, Training accuracy:0.731, Validation Loss: 8.731e-02, Validation Accuracy: 0.735, Time: 133.348, Learning Rate: 1.000e-03
